## 11.1 그래디언트 소실과 폭주 

### 11.1.1 글로럿과 He 초기화

파이썬은 기본적으로 글로럿 초기화를 사용한다고 한다. 만약 바꾸고 싶으면
```{.python}
keras.layers.Dense(10, activation = 'relu', kernel_initializer = 'he_normal')
```
로 해주면 된다

### 11.1.2 수렴하지 않는 활성화 함수

- LeakyReLU

이렇게 렐루 층을 만들고 적용하려는 층 뒤에 추가하면 된다.

``` {.python}
model = keras.models.Sequential([
    [...]
    keras.layers.Dense(10, kernel_initializer = 'he_normal',
    keras.layers.LeakyReLU(alpha = 0.2),
    [...]
    ])
```    

얘처럼 alpha = 0.2로 두면 PReLU가 된다고 함

- SELU

활성화함수를 바꾸고 커널초기화를 르쿤으로 바꾸면 된다고 함

```{.python}layer = keras.layer.Dense(10, activation = 'selu', kernel_initializer = 'lecun_normal')```

### 11.1.3 Batch Normalization 

아래와 같이 layers로 배치 정규화 층을 만들어주면 된다.

In [9]:
import tensorflow as tf
from tensorflow import keras
import pydot

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape = [28,28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation = 'elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation = 'elu', kernel_initializer='he_normal'),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation='softmax')
    
])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 300)               1200      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
batch_normalization_2 (Batch (None, 100)               400       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1

In [12]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [14]:
model.layers[1]

## 11.2 Transfer Learning 

### 11.2.1 케라스를 사용한 전이학습

#model 불러오기
```{.python}
model_a = keras.models.load_model("my_keras_model.h5")
```

#model 불러서 넣기
```{.python}
model_b_on_a = keras.models.Sequential(model_a.layers[:-1])
model_b_on_a.add(keras.layers.Dense(1, activation = 'sigmoid'))
```

model_b_on_a 를 훈련하면 model_a도 영향을 받는다고 한다. 그러므로 층을 재사용 하기 전에 model_a를 복제해야한다

#clone만들기
```{.python}
model_a_clone = keras.models.clone_model(model_a)
model_a_clone.set_weights(model_a.get_weights())
```

## 11.3 Optimizer

### 11.3.1 Momentum

```{.python}
optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9)
```

모멘텀 파라미터인 베타가 추가되긴 하지만, 보통 0.9에서 좋은 성능을 낸다고 한다.



### 11.3.2 네스테로프 가속 경사

```{.python}
optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, nesterov = True)
```


### 11.3.3 AdaGrad

이는 신경망에서 사용하면 안된다고 한다. 학습률이 너무 감소되어 global minima에 도달하기 전에 멈춰버리기 때문

### 11.3.4 RMSProp

```{.python}
optimizer = keras.optimizers.RMSprop(lr = 0.001, rho = 0.9)
```


### 11.3.5 Adam과 변종들

```{.python}
optimizer = keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999)
```
beta1은 모멘텀 감쇠 파라미터, beta2는 스케일 감쇠 파라미터라고 한다.